# Deploy Gluon-trained model with Model Server

In [ ]:
!sudo yum install protobuf-compiler libprotoc-dev

In [ ]:
!pip install mxnet-model-server

In [1]:
import mxnet

In [2]:
mxnet.__version__

'1.1.0'

In [5]:
a = mxnet.nd.ones(1) 

In [8]:
a.asnumpy()[0]

1.0

In [41]:
%%writefile crepe_cn_service.py


from mms.model_service.mxnet_model_service import MXNetBaseService
from mxnet.gluon.data import ArrayDataset
import numpy as np
import mxnet as mx

ALPHABET = list("abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+ =<>()[]{}")
ALPHABET_INDEX = {letter: index for index, letter in enumerate(ALPHABET)} # { a: 0, b: 1, etc}
FEATURE_LEN = 1014 # max-length in characters for one document

# Encode the text for processing
def encode(text):
    encoded = np.zeros([len(ALPHABET), FEATURE_LEN], dtype='float32')
    review = text.lower()[::-1]
    i = 0
    for letter in text:
        if i >= FEATURE_LEN:
            break;
        if letter in ALPHABET_INDEX:
            encoded[ALPHABET_INDEX[letter]][i] = 1
        i += 1
    return encoded

class TextClassicationService(MXNetBaseService):
    MAX_BATCH_SIZE = 128
    
    def _preprocess(self, data):
        assert (len(data) <= MAX_BATCH_SIZE), 'Too many inputs {}'.format(len(data))
        encoded_data = [encode(d) for d in data]
        return map(mx.nd.array, encoded_data)

    def _postprocess(self, data):
        indices = nd.argmax(data, axis=1)
        return [self.labels[index] for index in indices]

Writing crepe_cn_service.py


In [20]:
data = ["I am a review", "I am a second review", "You are a third review"]